In [16]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import poisson 
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import preprocessing

def sum_of_lower_upper(mat):
    r = len(mat)
    c = len(mat[0])
    diag = np.trace(mat)
    lower_sum = 0
    upper_sum = 0
    over_sum = 0
    under_sum = 0

    for i in range(r):
        for j in range(c):
            if (i<j):
                upper_sum += mat[i][j]

    for i in range(r):
        for j in range(c):
            if (j<i):
                lower_sum += mat[i][j]
    
    for i in range(r):
        for j in range(c):
            if (i+j>=3):
                over_sum += mat[i][j]
            else:
                under_sum += mat[i][j]

    return (upper_sum, lower_sum, diag, over_sum, under_sum)


## Gathering Lamda from season long performance

In [17]:
df1 = pd.read_csv('GatheredData/Full20xxFiles/Full2017.csv')
df2 = pd.read_csv('GatheredData/Full20xxFiles/Full2018.csv')
df3 = pd.read_csv('GatheredData/Full20xxFiles/Full2019.csv')
df4 = pd.read_csv('GatheredData/Full20xxFiles/Full2020.csv')
df5 = pd.read_csv('GatheredData/Full20xxFiles/Full2021.csv')
df6 = pd.read_csv('GatheredData/Full20xxFiles/Full2022.csv')
PRED = pd.read_csv('Prediction.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]

df1 = df1[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df3.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df3.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df4 = df4[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df4.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df4.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df5 = df5[['Date_x', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'AvgC>2.5', 'AvgC<2.5']]
df5.rename(columns = {'AvgC<2.5':'BbAv<2.5'}, inplace = True)
df5.rename(columns = {'AvgC>2.5':'BbAv>2.5'}, inplace = True)
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df6.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df6.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)

df1.rename(columns = {'Date_x':'Date'}, inplace = True)
df2.rename(columns = {'Date_x':'Date'}, inplace = True)
df3.rename(columns = {'Date_x':'Date'}, inplace = True)
df4.rename(columns = {'Date_x':'Date'}, inplace = True)
df5.rename(columns = {'Date_x':'Date'}, inplace = True)

Season = pd.read_csv('GatheredData/Full20xxFiles/Full2023.csv')
Season = Season[['HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
Season = Season.iloc[38:]

DF = pd.concat([df1, df2, df3, df4, df5, df6, Season], ignore_index=True)


DF = DF[ DF['HM'] != 0] 
DF = DF[ DF['HM'] != 1] 
DF = DF[ DF['HM'] != 2] 
DF = DF[ DF['HM'] != 3]
subset = DF[['Date', 'HTeam', 'ATeam']].copy()
extra_stats = DF.copy()
extra_pred = pred.copy()

DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
DF = pd.concat([DF, pred], ignore_index=True)
try:
    DF['HxG'] = DF['HxG'].div(DF['HM'])
    DF['HG'] = DF['HG'].div(DF['HM'])
    DF['HGA'] = DF['HGA'].div(DF['HM'])
    DF['HxGA'] = DF['HxGA'].div(DF['HM'])
    DF['AxG'] = DF['AxG'].div(DF['AM'])
    DF['AG'] = DF['AG'].div(DF['AM'])
    DF['AxGA'] = DF['AxG'].div(DF['AM'])
    DF['AGA'] = DF['AGA'].div(DF['AM'])
    pred['HxG'] = pred['HxG'].div(pred['HM'])
    pred['HG'] = pred['HG'].div(pred['HM'])
    pred['HGA'] = pred['HGA'].div(pred['HM'])
    pred['HxGA'] = pred['HxGA'].div(pred['HM'])
    pred['AxG'] = pred['AxG'].div(pred['AM'])
    pred['AG'] = pred['AG'].div(pred['AM'])
    pred['AxGA'] = pred['AxG'].div(pred['AM'])
    pred['AGA'] = pred['AGA'].div(pred['AM'])

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0

HGoals = []
AGoals = []
    
for i, x in DF.iloc[:cut].iterrows():
    a=x['Result'].split('\'')
    HGoals.append(a[1])
    AGoals.append(a[3])
    
DF=np.array(DF)
DFnorm = preprocessing.normalize(DF[:, 0:21], axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF


y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHome= reg1.predict(pred)
LamdaAway= reg2.predict(pred)

## Gathering Lamda from 2 month form performance

In [18]:
df1 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2017Monthly.csv')
df2 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2018Monthly.csv')
df3 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2019Monthly.csv')
df4 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2020Monthly.csv')
df5 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2021Monthly.csv')
df6 = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2022Monthly.csv')
pred = PRED.copy()
#Keeping bookmakers odds
Odds = PRED.copy()
Odds = PRED[['HTeam', 'ATeam','B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]

df1 = df1[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df2 = df2[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
df3 = df3[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df4 = df4[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df5 = df5[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df6 = df6[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
df6.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
df6.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)

SeasonForm = pd.read_csv('GatheredData/Epl20xxMonthlyFiles/Epl2023Monthly.csv')
    

try:
    SeasonForm = SeasonForm[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'BbAv>2.5', 'BbAv<2.5']]
except KeyError:
    SeasonForm = SeasonForm[['Date', 'HxG', 'HM', 'HTeam', 'ATeam', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'Result', 'B365H', 'B365D', 'B365A', 'B365>2.5', 'B365<2.5']]
    SeasonForm.rename(columns = {'B365<2.5':'BbAv<2.5'}, inplace = True)
    SeasonForm.rename(columns = {'B365>2.5':'BbAv>2.5'}, inplace = True)
    
SeasonForm = SeasonForm.iloc[38:]

DF = pd.concat([df1, df2, df3, df4, df5, df6, SeasonForm], ignore_index=True)
merged = pd.merge(subset, DF, on=['Date', 'HTeam', 'ATeam'], how='left')
# resulting dataframe will have columns of B where the triplet of A['a', 'b', 'c'] exists in B
DF = merged[DF.columns]
DF['PTSDIF/M']=DF['HPTS/M']-DF['APTS/M']
pred['PTSDIF/M']=pred['HPTS/M']-pred['APTS/M']
DF = DF[['HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
pred = pred[[ 'HxG', 'HM', 'HG', 'HGA', 'HxGA', 'HDC', 'HODC', 'HPPDA', 'HOPPDA', 'HPTS/M', 'AM', 'AG', 'AGA', 'AxG', 'AxGA', 'ADC', 'AODC', 'APPDA', 'AOPPDA', 'APTS/M', 'PTSDIF/M', 'Result']]
cut = DF.shape[0]
DF = pd.concat([DF, pred], ignore_index=True)


try:
    DF['HxG'] = DF['HxG'].div(DF['HM']).fillna(0)
    DF['HG'] = DF['HG'].div(DF['HM']).fillna(0)
    DF['HGA'] = DF['HGA'].div(DF['HM']).fillna(0)
    DF['HxGA'] = DF['HxGA'].div(DF['HM']).fillna(0)
    DF['AxG'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AG'] = DF['AG'].div(DF['AM']).fillna(0)
    DF['AxGA'] = DF['AxG'].div(DF['AM']).fillna(0)
    DF['AGA'] = DF['AGA'].div(DF['AM']).fillna(0)
    pred['HxG'] = pred['HxG'].div(pred['HM']).fillna(0)
    pred['HG'] = pred['HG'].div(pred['HM']).fillna(0)
    pred['HGA'] = pred['HGA'].div(pred['HM']).fillna(0)
    pred['HxGA'] = pred['HxGA'].div(pred['HM']).fillna(0)
    pred['AxG'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AG'] = pred['AG'].div(pred['AM']).fillna(0)
    pred['AxGA'] = pred['AxG'].div(pred['AM']).fillna(0)
    pred['AGA'] = pred['AGA'].div(pred['AM']).fillna(0)

except ZeroDivisionError:
    DF['HxG'] = 0
    DF['HG'] = 0
    DF['HGA'] = 0
    DF['HxGA'] = 0
    DF['AxG'] = 0
    DF['AG'] = 0
    DF['AxGA'] = 0
    DF['AGA'] = 0

HGoals = []
AGoals = []
for i, x in DF.iloc[:cut].iterrows():
    a=x['Result'].split('\'')
    HGoals.append(a[1])
    AGoals.append(a[3])

DF=np.array(DF)

DFnorm = preprocessing.normalize(DF[:, 0:21].astype('float64'), axis=0)
pred = DFnorm[cut:]
DF = DFnorm[:cut]

x=DF

y1=np.array(HGoals)
y2=np.array(AGoals)

reg1 = LinearRegression().fit(x, y1)
reg2 = LinearRegression().fit(x, y2)

LamdaHomeForm= reg1.predict(pred)
LamdaAwayForm= reg2.predict(pred)

KeyError: "['BbAv>2.5', 'BbAv<2.5'] not in index"

In [ ]:
LamdaAwayFinal = [(LamdaAway[i] * 0.4 + LamdaAwayForm[i] * 0.6) for i in range(len(LamdaAway))]
LamdaHomeFinal = [(LamdaHome[i] * 0.4 + LamdaHomeForm[i] * 0.6) for i in range(len(LamdaHome))]

print(LamdaHomeFinal)
print(LamdaAwayFinal)

[1.4926046543078029, 4.209859936771937, 2.727016594822108, 2.586338019722675, 2.9599191155607536, 4.692630610387585, 2.6986159024006366, 2.760670703369709, 2.293835365473037, 4.260849505762339, 2.8065959774234646, 3.413776233966545, 3.127009173011323, 3.102776764261201, 3.302164418871876, 4.4362912112560124, 2.530683359013339, 0.8692304489221061, 3.144246619904685]
[0.48271657570512816, 3.5300538055912507, 1.7501190895575345, 1.51378963460985, 2.236108204608467, 1.3252361424223142, 1.4386564648521705, 1.9408211488265623, 1.5397881300154264, 0.8943711566224661, 1.6260270910342718, 0.5597506471521967, 1.8860554480052576, 0.7459352647206305, 1.0444623677413596, 0.6966611259455473, 1.180810120288025, 2.7128956314364494, 0.7630177090359893]


In [ ]:
k = range(12)
PH = np.zeros((len(k), 1))
PA = np.zeros((1, len(k)))
PRED['P(Hwin)']= np.nan
PRED['P(Awin)']= np.nan
PRED['P(Draw)']= np.nan
PRED['P(Over)']= np.nan
PRED['P(Under)']= np.nan

Pie_list = []
for j, l in enumerate(LamdaHome):
    for i in k:
        PH[i, 0] = (math.exp(-l)*l**k[i]) /math.factorial(k[i])
        PA[0, i] = (math.exp(-LamdaAway[j])*LamdaAway[j]**k[i]) /math.factorial(k[i])
    Pscore = np.matmul(PH, PA) 
    df = pd.DataFrame(Pscore, columns = ['0','1','2', '3', '4', '5', '6', '7', '8', '9', '10', '11'])
    PRED.loc[j,'P(Awin)'], PRED.loc[j,'P(Hwin)'], PRED.loc[j,'P(Draw)'], PRED.loc[j,'P(Over)'], PRED.loc[j,'P(Under)']  = sum_of_lower_upper(Pscore)
    PRED['P(Over)'] = 1 - PRED['P(Under)']
    df = df.iloc[0:5, 0:5]
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    
    G = []
    for i in range(0, 5):
        for j in range(0, 5):
            G.append(df.loc[i][j])

            
    Pie_Df = pd.DataFrame(G, columns = ['P(score)'])
    Pie_Df['score']= ['0 - 0',
                      '0 - 1',
                      '0 - 2',
                      '0 - 3',
                      '0 - 4',
                      '1 - 0',
                      '1 - 1',
                      '1 - 2',
                      '1 - 3',
                      '1 - 4',
                      '2 - 0',
                      '2 - 1',
                      '2 - 2',
                      '2 - 3',
                      '2 - 4',
                      '3 - 0',
                      '3 - 1',
                      '3 - 2',
                      '3 - 3',
                      '3 - 4',
                      '4 - 0',
                      '4 - 1',
                      '4 - 2',
                      '4 - 3',
                      '4 - 4']
    Pie_Df.loc[Pie_Df["P(score)"] < 0.02, "score"] = 'Other'
    Pie_Df = Pie_Df.sort_values(by=['P(score)'], ascending=False)
    Pie_list.append(Pie_Df)
    

PRED = PRED[['HTeam', 'ATeam', 'P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(PRED)

Final_Odds = pd.concat([PRED, Odds], axis=0)
Final_Odds['Type'] = np.nan
Final_Odds['Match'] = np.nan
Final_Odds = Final_Odds.reset_index(drop = 'True')
Final_Odds.loc[Final_Odds['B365H'].isnull(), 'Type'] = 'Predicted'
Final_Odds.loc[Final_Odds['P(Hwin)'].isnull(), 'Type'] = 'Bookmaker'

for i, x in enumerate(Final_Odds['Match']):
    Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])

li  = Final_Odds.index[Final_Odds['Type'] == 'Bookmaker'].tolist()
for i in li:
    Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
    Final_Odds['P(Awin)'][i] = 1/Final_Odds.iloc[i]['B365A']
    Final_Odds['P(Draw)'][i] = 1/Final_Odds.iloc[i]['B365D']
    Final_Odds['P(Over)'][i] = 1/Final_Odds.iloc[i]['BbAv>2.5']
    Final_Odds['P(Under)'][i] = 1/Final_Odds.iloc[i]['BbAv<2.5']

Final_Odds = Final_Odds[['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Under)', 'P(Over)', 'Type', 'Match']].copy()

,HTeam,ATeam,P(Hwin),P(Draw),P(Awin),P(Under),P(Over)
0,Nottingham Forest,Sheffield United,0.730,0.229,0.041,0.746,0.254
1,Wolverhampton Wanderers,Brighton,0.121,0.101,0.775,0.032,0.968
2,Liverpool,Bournemouth,0.431,0.203,0.366,0.221,0.779
3,Fulham,Brentford,0.371,0.209,0.420,0.250,0.750
4,Tottenham,Manchester United,0.333,0.182,0.485,0.131,0.869
5,Manchester City,Newcastle United,0.450,0.181,0.370,0.117,0.883
6,Aston Villa,Everton,0.276,0.212,0.512,0.310,0.690
7,West Ham,Chelsea,0.355,0.199,0.446,0.203,0.797
8,Crystal Palace,Arsenal,0.504,0.204,0.292,0.261,0.739
9,Chelsea,Luton,0.783,0.115,0.101,0.126,0.874


/tmp/ipykernel_13687/3739398557.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['Match'][i] = str(Final_Odds['HTeam'][i] + '-' + Final_Odds['ATeam'][i])
/tmp/ipykernel_13687/3739398557.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Hwin)'][i] = 1/Final_Odds.iloc[i]['B365H']
/tmp/ipykernel_13687/3739398557.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Odds['P(Awin)'][i] = 1/Final_Odds.ilo

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.update_layout(width=1500, height=1000)
fig.add_trace(
    go.Bar(name='Predicted Home Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Hwin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Home Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Hwin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Draw',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Draw)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Draw',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Draw)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Predicted Away Win',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Awin)'],
           marker_color='rgb(141,211,199)'),
    row=1, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Away Win',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Awin)'],
           marker_color='rgb(82, 106, 131)'),
    row=1, col=1)


fig.add_trace(
    go.Bar(name='Predicted Over',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Over)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Over',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Over)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Predicted Under',
           x=Final_Odds[Final_Odds['Type']=='Predicted']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'],
           marker_color='rgb(141,211,199)'),
    row=2, col=1)

fig.add_trace(
    go.Bar(name='Bookmaker Under',
           x=Final_Odds[Final_Odds['Type']=='Bookmaker']['Match'],
           y=Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'],
           marker_color='rgb(82, 106, 131)'),
    row=2, col=1)


fig.show()


In [ ]:
# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'xy'}]], column_widths=[0.33, 0.67])
fig.update_layout(width=1500, height=650)
# create a list to store the buttons
buttons = []
categories = ['Home', 'Draw', 'Away', 'Over', 'Under']

# loop through each match and create two traces for Bookmaker odds and predicted values
for match_index, match in enumerate(Final_Odds['Match'].unique().tolist()):

    # create the trace for Bookmaker odds
    trace_bookmaker = go.Scatterpolar(
      r=Final_Odds[(Final_Odds['Type']=='Bookmaker')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)']].values.tolist()[0],
      theta=categories,
      fill='toself',
      name='Bookmaker Odds',
      marker = dict(color = 'rgb(82, 106, 131)' ),
      visible=False if match_index!=0 else True,
      hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Bookmaker Odds'
)
    fig.add_trace(trace_bookmaker, row=1, col=1)
    # create the trace for predicted values
    trace_predicted = go.Scatterpolar(
        r= Final_Odds[(Final_Odds['Type']=='Predicted')  & (Final_Odds['Match']==match)][['P(Hwin)', 'P(Draw)', 'P(Awin)', 'P(Over)', 'P(Under)']].values.tolist()[0],
        theta=categories,
        fill='toself',
        name='Predicted Odds',
        marker = dict(color ='rgb(141,211,199)'), #'rgb(217, 175, 107)'
        visible=False if match_index!=0 else True,
        hovertemplate='%{theta}<br>Prob: %{r:.2f}<br>Predicted Odds'
  )
    fig.add_trace(trace_predicted, row=1, col=1)
    
    trace_goals = go.Bar(
        x=Pie_list[match_index]['score'],
        y=Pie_list[match_index]['P(score)'],
        visible=False if match_index!=0 else True,
        marker=dict(color=Pie_list[match_index]['P(score)'], colorscale='darkmint'),
        name='Propable Scoreline',
        hovertemplate='Score %{x}<br>Prob: %{y:.3f}'
    )
    fig.add_trace(trace_goals, row=1, col=2)
    
    # create the dictionary for the button
    button_dict = dict(
        method="restyle",
        args=[{"visible": [False] * (len(Final_Odds['Match'].unique().tolist()) * 3)}], # set all traces to false initially
        label=match
    )
    
    # set visibility to true for the corresponding traces
    button_dict["args"][0]["visible"][3*match_index] = True # Bookmaker odds trace
    button_dict["args"][0]["visible"][3*match_index+1] = True # Predicted trace
    button_dict["args"][0]["visible"][3*match_index+2] = True # Goal trace
    
    # set visibility to true for the corresponding traces for the first match
    if match_index == 0:
        button_dict["args"][0]["visible"][0] = True # Bookmaker odds trace
        button_dict["args"][0]["visible"][1] = True # Predicted trace
        button_dict["args"][0]["visible"][2] = True
        
    # append the button to the list
    buttons.append(button_dict)


# update the layout with the buttons
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        buttons=buttons,
        active=0, # set the initial active button
        x=0.1,
        y=1.2,
        direction="down"
    )],
    
)

# show the plot
fig.show()


In [ ]:
Pont = 5
bank = 50
Final = Final_Odds[Final_Odds['Type']=='Predicted'][['Match']]
listPred = Final_Odds[Final_Odds['Type']=='Predicted']['P(Under)'].tolist()
listBook = Final_Odds[Final_Odds['Type']=='Bookmaker']['P(Under)'].tolist()

diff = []
for i, p in enumerate(listPred):
    diff.append(listPred[i]-listBook[i])

Final['UnderBet'] = np.nan
Final['UnderBet_Kelly'] = np.nan
for i, d in enumerate(diff):
    if (0.7>=listPred[i]>=0.35) and (d>=0.001):
        Final['UnderBet'][i] = max(Pont*10*(d),0)
        Final['UnderBet_Kelly'][i] = max(0, bank*(listPred[i]- (1-listPred[i])/(1/listBook[i] -1)))
    else:
        Final['UnderBet'][i] = 0
        Final['UnderBet_Kelly'][i] = 0
        
############

    
display(Final)

/tmp/ipykernel_13687/2079959016.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13687/2079959016.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13687/2079959016.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13687/2079959016.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

,Match,UnderBet,UnderBet_Kelly
0,Nottingham Forest-Sheffield United,0.000,0.000
1,Wolverhampton Wanderers-Brighton,0.000,0.000
2,Liverpool-Bournemouth,0.000,0.000
3,Fulham-Brentford,0.000,0.000
4,Tottenham-Manchester United,0.000,0.000
5,Manchester City-Newcastle United,0.000,0.000
6,Aston Villa-Everton,0.000,0.000
7,West Ham-Chelsea,0.000,0.000
8,Crystal Palace-Arsenal,0.000,0.000
9,Chelsea-Luton,0.000,0.000
